In [1]:
'''This script demonstrates how to build a variational autoencoder
with Keras and deconvolution layers.
# Reference
- Auto-Encoding Variational Bayes
  https://arxiv.org/abs/1312.6114
'''
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import keras
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Conv1D
from keras.layers import Conv2D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import config
import os, numpy as np, pandas, sklearn, scipy.signal as signal
import mido
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# local libs
import config, models, functions
from data import data, midi, midi_generators as g
from utils import io, models_io, utils, plot

 Context :: namedtuple(
[ max_t = float
, dt = float
, n_instances = int
, note_length = int
, bpm = float
, tempo = float
, ticks_per_beat = int
]



In [4]:
context = data.init()
n = 2
multiTrack = True
context, x_train, labels = data.import_data(data.init(), n, multiTrack=multiTrack)

Setting up params

max min f 25.0 0.3333333333333333
 >> Context(max_t=3.0, dt=0.02, n_instances=150, note_length=0.03, bpm=120.0, tempo=500000, ticks_per_beat=480)
Setting up params

max min f 25.0 0.3333333333333333
 >> Context(max_t=3.0, dt=0.02, n_instances=150, note_length=0.03, bpm=120.0, tempo=500000, ticks_per_beat=480)
Importing midi-data

 [INFO] : 
 |  reading file: ../datasets/examples/01 16th Snare.mid
 [INFO] : 
 |  reading file: ../datasets/examples/01 8th Cym.mid

Encoding midi-data
 [<midi file '../datasets/examples/01 16th Snare.mid' type 0, 1 tracks, 182 messages>, <midi file '../datasets/examples/01 8th Cym.mid' type 0, 1 tracks, 68 messages>]
> -> multi-track = True
<meta message track_name name='01 16th Snare\x00' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 no

In [5]:
n = 100
min_f = 3
max_f = 15
# x_train = g.gen_data(context, n, max_f=max_f, min_f=min_f)
# x_train = g.gen_data_complex(context, n, max_f=max_f, min_f=min_f, multiTrack=multiTrack)
x_train = g.gen_data_complex(context, n, max_f=max_f, min_f=min_f, 
    n_polyrythms=1,
    n_channels=midi.N_NOTES,
    multiTrack=multiTrack)

note_on channel=0 note=62 velocity=127 time=0.008333333333333333
note_on channel=0 note=60 velocity=127 time=0.01875
note_off channel=0 note=62 velocity=127 time=0.0010416666666666667
note_off channel=0 note=60 velocity=127 time=0.01875
note_on channel=0 note=61 velocity=127 time=0.019791666666666666
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.008333333333333333
note_on channel=0 note=60 velocity=127 time=0.0020833333333333333
note_off channel=0 note=62 velocity=127 time=0.017708333333333333
note_off channel=0 note=60 velocity=127 time=0.0020833333333333333
note_on channel=0 note=61 velocity=127 time=0.022916666666666665
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.008333333333333333
note_on channel=0 note=62 velocity=127 time=0.014583333333333334
note_off channel=0 note=60 velocity=127 time=0.00625
note_off channel=0 note=62 velocity=127 time=0.0145833

note_on channel=0 note=61 velocity=127 time=0.020833333333333332
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.0010416666666666667
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.026041666666666668
note_off channel=0 note=60 velocity=127 time=0.019791666666666666
note_on channel=0 note=61 velocity=127 time=0.009375
note_on channel=0 note=62 velocity=127 time=0.005208333333333333
note_off channel=0 note=61 velocity=127 time=0.014583333333333334
note_off channel=0 note=62 velocity=127 time=0.005208333333333333
note_on channel=0 note=62 velocity=127 time=0.06041666666666667
note_on channel=0 note=60 velocity=127 time=0
note_on channel=0 note=61 velocity=127 time=0.009375
note_off channel=0 note=62 velocity=127 time=0.010416666666666666
note_off channel=0 note=60 velocity=127 time=0
note_off channel=0 note=61 velocity=127 time=0.009375
note_on channel=0 note=62 

note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.0375
note_on channel=0 note=61 velocity=127 time=0.010416666666666666
note_off channel=0 note=60 velocity=127 time=0.009375
note_off channel=0 note=61 velocity=127 time=0.010416666666666666
note_on channel=0 note=62 velocity=127 time=0.046875
note_on channel=0 note=60 velocity=127 time=0.008333333333333333
note_off channel=0 note=62 velocity=127 time=0.011458333333333333
note_off channel=0 note=60 velocity=127 time=0.008333333333333333
note_on channel=0 note=60 velocity=127 time=0.065625
note_on channel=0 note=61 velocity=127 time=0.00625
note_off channel=0 note=60 velocity=127 time=0.013541666666666667
note_off channel=0 note=61 velocity=127 time=0.00625
note_on channel=0 note=62 velocity=127 time=0.013541666666666667
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.026041666666666668
note_off channel=0 note=60 v

note_on channel=0 note=61 velocity=127 time=0.08229166666666667
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.027083333333333334
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=61 velocity=127 time=0.034375
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.046875
note_off channel=0 note=60 velocity=127 time=0.019791666666666666
note_on channel=0 note=61 velocity=127 time=0.015625
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.004166666666666667
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=61 velocity=127 time=0.058333333333333334
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=61 velocity=127 time=0.08229166666666667
note_on channel=0 note=62 velocity=127 time=0.0010416666

note_off channel=0 note=62 velocity=127 time=0.014583333333333334
note_on channel=0 note=61 velocity=127 time=0.004166666666666667
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.022916666666666665
note_on channel=0 note=62 velocity=127 time=0.01875
note_off channel=0 note=60 velocity=127 time=0.0010416666666666667
note_off channel=0 note=62 velocity=127 time=0.01875
note_on channel=0 note=60 velocity=127 time=0.042708333333333334
note_off channel=0 note=60 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.0020833333333333333
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.03958333333333333
note_off channel=0 note=60 velocity=127 time=0.019791666666666666
note_on channel=0 note=61 velocity=127 time=0.004166666666666667
note_on channel=0 note=62 velocity=127 time=0.0020833333333333333
note_off channel=0 note=61 velocity=127 ti

...

note_off channel=0 note=61 velocity=127 time=0.0125
note_on channel=0 note=61 velocity=127 time=0.053125
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.00625
note_on channel=0 note=60 velocity=127 time=0
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_off channel=0 note=60 velocity=127 time=0
note_on channel=0 note=61 velocity=127 time=0.027083333333333334
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.04479166666666667
note_on channel=0 note=61 velocity=127 time=0.009375
note_on channel=0 note=62 velocity=127 time=0.008333333333333333
note_off channel=0 note=60 velocity=127 time=0.003125
note_off channel=0 note=61 velocity=127 time=0.009375
note_off channel=0 note=62 velocity=127 time=0.008333333333333333
note_on channel=0 note=61 velocity=127 time=0.04479166666666667
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
n

note_on channel=0 note=60 velocity=127 time=0.05
note_on channel=0 note=62 velocity=127 time=0.007291666666666667
note_on channel=0 note=61 velocity=127 time=0.011458333333333333
note_off channel=0 note=60 velocity=127 time=0.0010416666666666667
note_off channel=0 note=62 velocity=127 time=0.007291666666666667
note_off channel=0 note=61 velocity=127 time=0.011458333333333333
note_on channel=0 note=62 velocity=127 time=0.04479166666666667
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.007291666666666667
note_off channel=0 note=60 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.030208333333333334
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=61 velocity=127 time=0.027083333333333334
note_on channel=0 note=60 velocity=127 time=0.014583333333333334
note_off channel=0 note=61 velocity=127 time=0.00625
note_on channel=0 note=62 velocity=127 time=0.0

note_off channel=0 note=61 velocity=127 time=0.011458333333333333
note_off channel=0 note=60 velocity=127 time=0.0020833333333333333
note_on channel=0 note=62 velocity=127 time=0.042708333333333334
note_on channel=0 note=61 velocity=127 time=0.0125
note_on channel=0 note=60 velocity=127 time=0.005208333333333333
note_off channel=0 note=62 velocity=127 time=0.0020833333333333333
note_off channel=0 note=61 velocity=127 time=0.0125
note_off channel=0 note=60 velocity=127 time=0.005208333333333333
note_on channel=0 note=62 velocity=127 time=0.03854166666666667
note_on channel=0 note=61 velocity=127 time=0.013541666666666667
note_off channel=0 note=62 velocity=127 time=0.00625
note_on channel=0 note=60 velocity=127 time=0.003125
note_off channel=0 note=61 velocity=127 time=0.010416666666666666
note_off channel=0 note=60 velocity=127 time=0.009375
note_on channel=0 note=62 velocity=127 time=0.03333333333333333
note_on channel=0 note=61 velocity=127 time=0.014583333333333334
note_off channel=

note_on channel=0 note=61 velocity=127 time=0.027083333333333334
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.0125
note_on channel=0 note=60 velocity=127 time=0.005208333333333333
note_off channel=0 note=62 velocity=127 time=0.014583333333333334
note_off channel=0 note=60 velocity=127 time=0.005208333333333333
note_on channel=0 note=61 velocity=127 time=0.05
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.0625
note_on channel=0 note=61 velocity=127 time=0.005208333333333333
note_on channel=0 note=62 velocity=127 time=0.008333333333333333
note_off channel=0 note=60 velocity=127 time=0.00625
note_off channel=0 note=61 velocity=127 time=0.005208333333333333
note_off channel=0 note=62 velocity=127 time=0.008333333333333333
note_on channel=0 note=62 ve

 |>  2.9947916666666687
 |>  150
note_on channel=0 note=60 velocity=127 time=0.0125
note_off channel=0 note=60 velocity=127 time=0.019791666666666666
note_on channel=0 note=61 velocity=127 time=0.06354166666666666
note_on channel=0 note=60 velocity=127 time=0.019791666666666666
note_off channel=0 note=61 velocity=127 time=0
note_off channel=0 note=60 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.051041666666666666
note_off channel=0 note=62 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.0125
note_off channel=0 note=60 velocity=127 time=0.019791666666666666
note_on channel=0 note=61 velocity=127 time=0.0125
note_off channel=0 note=61 velocity=127 time=0.019791666666666666
note_on channel=0 note=60 velocity=127 time=0.05
note_off channel=0 note=60 velocity=127 time=0.019791666666666666
note_on channel=0 note=62 velocity=127 time=0.051041666666666666
note_on channel=0 note=61 velocity=127 time=0.01354166666666666

In [6]:
n = 10
total = 1000 * 1
x_test = x_train[n:]
x_train = np.concatenate([x_train[:n] for _ in range(int(total/n))])

In [7]:
img_dims = (28,28,1)
img_dims = x_train[0].shape
img_dims

(150, 3)

In [42]:
# input image dimensions
# no cols
img_rows, img_chns = img_dims
# number of convolutional filters to use
filters = 64
# convolution kernel size
num_conv = 3

batch_size = 100

original_img_size = (img_rows, img_chns)

latent_dim = 2
intermediate_dim = 128
epsilon_std = 1.0
epochs = 5

In [43]:
encoder_input = Input(shape=original_img_size)

timesteps = original_img_size[-2]
notes = original_img_size[-1]
    
    # Convolve & Pool
h = encoder_input
h = Conv1D(32, kernel_size=2, strides=2, activation='relu', padding='same')(h)
# h = Conv1D(32, kernel_size=2, strides=1, activation='relu', padding='same')(h)
h = Conv1D(64, kernel_size=2, strides=1, activation='relu', padding='same')(h)

    # Dense layers
h = Flatten()(h)
h = Dense(intermediate_dim, activation='relu')(h)
#     h = Dense(intermediate_dim, activation='relu')(h)
#     h = Dense(intermediate_dim, activation='relu')(h)    
#     h = Dropout(0.1)(h) # uncomment when using larger batches
#     h = Dense(intermediate_dim, activation='relu')(h)
    # h = Dense(intermediate_dim, activation='relu', kernel_regularizer=k_reg, activity_regularizer=a_reg)(h)

    # Z Mean, Variance
z_mean = Dense(latent_dim, name='z_mean')(h) # , activation='relu'
z_log_var = Dense(latent_dim, name='z_log_var')(h) # , activation='relu'
    
encoder_output = [z_mean, z_log_var]

In [44]:
encoder_model = Model(encoder_input, encoder_output, name='encoder_model-')

In [45]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var) * epsilon

In [46]:
# z = Lambda(sampling)([z_mean, z_log_var])
z_input = encoder_model(encoder_input)
z_output = Lambda(sampling)(z_input)

In [59]:
# decoder_input = z_output
# h = decoder_input
    # :output_shape = (timesteps, channels, channels) || (batches, filters, timesteps, channels)
    # keras offers just Conv2DTranspose and not Conv1DTranspose
    # - use 2D images during upsampling :: (timesteps, notes, channels) => (timesteps, notes, filters)
    # - use 1D images to optimize reconstruction :: (timesteps, filters) => (timesteps, notes)
    
    # at the start of upsampling, the image-structure does not yet have to correspond to the goal structure 
    # ?TODO use y*y*y starting dims, may conv, and only then correct the structure (?)

timesteps = original_img_size[-2] # cols
notes = original_img_size[-1] # rows
channels = 1 # rgb

    # image_data_format = 'channels_last'
    # goal shape: (timesteps, notes, channels)
    # start with the 'reverse': lots of small imgs => few large img
m = 4 # multiplier for dims

p = 'valid'

In [60]:
output_shape = (channels, notes * m, timesteps)

# we instantiate these layers separately so as to reuse them later
decoders = []
decoders += [ Dense(intermediate_dim, activation='relu') ]
decoders += [ Dense(np.prod(output_shape[-3:]), activation='relu') ]
decoders += [ Reshape(output_shape[-3:]) ]

# Convolve & Pool

# Note that the kernel windows do not yet correspond to 'temporal' features, but rather to just spatial features
k = (3,2) # (2,1) :: (timesteps, notes)
s = (1,3)
timesteps_ = int(timesteps / 2)

decoders += [ Conv2DTranspose(timesteps_, kernel_size=k, strides=3, activation='relu', padding=p) ]
decoders += [ Conv2DTranspose(32, kernel_size=k, strides=3, activation='relu', padding=p) ]
# decoder += [Conv2DTranspose(32, kernel_size=k, strides=3, activation='relu', padding=p)]
decoders += [ Conv2DTranspose(16, kernel_size=k, strides=(3,1), activation='relu', padding=p)]
decoders += [ Conv2DTranspose(16, kernel_size=k, strides=3, activation='relu', padding=p)]

# 'end' of upsampling
decoders += [ Conv2D(1, kernel_size=(1,2), strides=1, activation='relu', padding=p) ]
# shape = ()
# decoder_reshape2 = Reshape(dims[0:2])
# #     h = Conv1D(16, kernel_size=2, strides=1, activation='relu', padding=p)(h)
# decoder_conv2 = Conv1D(notes * m, kernel_size=2, strides=2, activation='relu', padding=p)


# Mean (output)
#     h = Conv2D(1, kernel_size=1, strides=s, activation='sigmoid', padding='same')(h)
output_shape = (timesteps, notes)

decoders += [ Flatten()]
decoders += [ Dense(np.prod(output_shape), activation='sigmoid')]
decoders += [ Reshape(output_shape)]

In [61]:
def composition(ls, result=lambda x: x, verbose=False):
    # ls = list of functions (pipeline)
    # return :: function composition of ls
    # e.g.
    # this should succeed without errors
    # compose([(lambda x: x) for _ in range(1000000)])(1)
    for f in ls[1:]:
        print(f)
        result = f(result)
    return result

In [64]:
decoded = composition(decoders, z_output, True)

In [67]:
# instantiate VAE model
vae_input = encoder_input
vae_output = decoded
vae = Model(vae_input, vae_output)
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 150, 3)       0                                            
__________________________________________________________________________________________________
encoder_model- (Model)          [(None, 2), (None, 2 619428      input_2[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 2)            0           encoder_model-[1][0]             
                                                                 encoder_model-[1][1]             
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 1800)         5400        lambda_2[0][0]                   
__________

In [68]:
# Compute VAE loss
# no img_cols! no notes!
def vae_loss():
    # y_true, y_pred, z_mean, z_log_var, timesteps=150, notes=3, beta=1.
    xent_loss = timesteps * keras.metrics.binary_crossentropy(K.flatten(vae_input), K.flatten(vae_output))
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)
    return vae_loss

vae_loss = vae_loss()
vae.add_loss(vae_loss)
vae.compile(optimizer='rmsprop')
# vae.compile(optimizer=optimizer, loss=loss, metrics=metrics)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Output "reshape_14" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "reshape_14" during training.
  if sys.path[0] == '':


In [69]:
# train the VAE on MNIST digits
# (x_train, _), (x_test, y_test) = mnist.load_data()
x_test = x_train[-100:]
y_test = x_test

In [70]:
x_train.shape

(1000, 150, 3)

In [71]:
print('x_train.shape:', x_train.shape)

x_train.shape: (1000, 150, 3)


In [ ]:
history = vae.fit(x_train,
        shuffle=True,
        epochs=2,
        batch_size=batch_size,
        validation_data=(x_test, None))

Train on 1000 samples, validate on 100 samples
Epoch 1/2
 300/1000 [========>.....................] - ETA: 12s - loss: 104.9447

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
# build a model to project inputs on the latent space
encoder = Model(encoder_input, z_mean)

In [ ]:
x_test.shape

In [ ]:
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
x_test_encoded.shape

In [ ]:
type(x_test_encoded[0,0])

In [ ]:
y_test = range(x_test.shape[0])
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)

In [ ]:
# display a 2D plot of the digit classes in the latent space
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
plt.colorbar()
plt.show()

In [ ]:
# build a digit generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_hid_decoded = decoder_hid(decoder_input)
_up_decoded = decoder_upsample(_hid_decoded)
_reshape_decoded = decoder_reshape(_up_decoded)
_deconv_1_decoded = decoder_deconv_1(_reshape_decoded)
_deconv_2_decoded = decoder_deconv_2(_deconv_1_decoded)
_x_decoded_relu = decoder_deconv_3_upsamp(_deconv_2_decoded)
_x_decoded_mean_squash = decoder_mean_squash(_x_decoded_relu)
generator = Model(decoder_input, _x_decoded_mean_squash)

In [ ]:
# display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
# linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# to produce values of the latent variables z, since the prior of the latent space is Gaussian
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
        x_decoded = generator.predict(z_sample, batch_size=batch_size)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()